# Production Planning Problem Example with PuLP

cuOpt is NVIDIA's GPU accelerated solver that delivers massive speedups for real-world LP, MIP, and VRP workloads.

cuOpt seemlessly integrates with modeling languages. You can drop cuOpt into existing models built with PuLP and AMPL, with minimal refactoring. Let's take a look at an example solving a simple MIP problem with cuOpt.

To run this in Google Colab, download the notebook and upload it to Google Colab. Make sure you are running this on a T4 GPU.

If you are running this in the cuOpt container, you are good to go!


## 1. Install Dependencies

To make sure we are good to go, let's install PuLP and cuOpt.

__[PuLP](https://coin-or.github.io/pulp/)__ is a popular linear and mixed integer programming modeler written in Python.


If you are running this notebook in Google Colab, or elsewhere outside the container where cuOpt is not yet installed, uncomment the pip install command to install cuOpt.

In [ ]:
import subprocess
from IPython.display import display, HTML

def check_gpu():
    try:
        output = subprocess.check_output("nvidia-smi", shell=True).decode()
        display(HTML(f"""
        <div style="border:2px solid #4CAF50;padding:10px;border-radius:10px;background:#e8f5e9;">
            <h3>✅ GPU is enabled</h3>
            <pre>{output.splitlines()[2]}</pre>
        </div>
        """))
    except Exception:
        display(HTML("""
        <div style="border:2px solid red;padding:15px;border-radius:10px;background:#ffeeee;">
            <h3>⚠️ GPU not detected!</h3>
            <p>This notebook requires a <b>GPU runtime</b>.</p>
            
            <h4>If running in Google Colab:</h4>
            <ol>
              <li>Click on <b>Runtime → Change runtime type</b></li>
              <li>Set <b>Hardware accelerator</b> to <b>GPU</b></li>
              <li>Then click <b>Save</b> and <b>Runtime → Restart runtime</b>.</li>
            </ol>
            
            <h4>If running in Docker:</h4>
            <ol>
              <li>Ensure you have <b>NVIDIA Docker runtime</b> installed (<code>nvidia-docker2</code>)</li>
              <li>Run container with GPU support: <code>docker run --gpus all ...</code></li>
              <li>Or use: <code>docker run --runtime=nvidia ...</code> for older Docker versions</li>
              <li>Verify GPU access: <code>docker run --gpus all nvidia/cuda:12.0.0-base-ubuntu22.04 nvidia-smi</code></li>
            </ol>
            
            <p><b>Additional resources:</b></p>
            <ul>
              <li><a href="https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html" target="_blank">NVIDIA Container Toolkit Installation Guide</a></li>
            </ul>
        </div>
        """))

check_gpu()

In [ ]:
!pip install pulp==3.2.0

In [ ]:
# # Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

#!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu12

## 2. Problem Setup

Let's consider the following problem:

A factory produces two products (x₁ and x₂) with the following constraints:  
- Profit: \$20 per unit of x₁, \$120 per unit of x₂  
- Resources:  
  - Material: 3 units/kg per x₁, 2 units/kg per x₂ (max 240 kg available)  
  - Labor: 2 hours per x₁, 4 hours per x₂ (max 180 hours available)  
- Special machine: Optional \$1000 fixed cost to enable production of x₂ (requires minimum 10 units of x₂ if used)

Key Features:  
1. Mixed variables:  
   - Integer variables for product quantities (x₁, x₂)  
   - Binary variable for machine activation (y)  

2. Conditional logic:  
   - The constraint `3*x1 + 2*x2 <= 240` correlates to the cost of materials
   - The constraint `2*x1 + 4*x2 <= 180 ` correlates to the cost of labor
   - The constraint `x2 >= 5*y` enforces that if the machine is used (y=1), at least 5 units of x₂ must be produced.  
   - The constraints `x1 >= 1` and `x2 >= 1` prevent trivial solutions, enforcing that we have both x1 and x2 in the solution.


3. Cost-benefit tradeoff:  
   The $1000 machine cost in the objective function creates a break-even analysis challenge.  

This formulation demonstrates how MIP models can handle both discrete decisions (machine usage) and continuous production quantities while optimizing complex business decisions.


In [ ]:
from pulp import *

# Define the problem
problem = LpProblem("Production_Planning", LpMaximize)

# Decision variables
x1 = LpVariable('x1', lowBound=0, cat='Integer')  # Product 1 units
x2 = LpVariable('x2', lowBound=0, cat='Integer')  # Product 2 units
y = LpVariable('y', cat='Binary')  # Machine usage flag

# Objective function: Maximize profit
problem += 20.0*x1 + 120.0*x2 + 1000.0*y, "Total_Profit"

# Constraints
problem += 3.0*x1 + 2.0*x2 <= 240.0, "Material_limit_x2"
problem += 2.0*x1 + 4.0*x2 <= 180.0, "Labor_limit_x2"
problem += x2 >= 5.0*y, "Minimum_x₂_if_machine_used"
problem += x1 >= 1.0, "Prevent_trivial_solution_x1"
problem += x2 >= 1.0, "Prevent_trivial_solution_x2"



## 3. Problem Solution

PuLP calls on the cuOpt solver, which finds the optimal values of x1, x2, and y that maximize the profit while satisfying the constraints.

In [ ]:

# Solve the problem using CUOPT
problem.solve(CUOPT(msg=0))

# Print results
print("Status:", LpStatus[problem.status])
print("x1 =", round(x1.varValue))
print("x2 =", round(x2.varValue))
print("y =", round(y.varValue))
print("Total Profit =", round(value(problem.objective)))